### Import libraries

In [1]:
import textract as tx
import spacy
from spacy import displacy
import en_core_web_sm
import pandas as pd
import numpy as np
import textract
import re
import tensorflow as tf
from transformers import BertTokenizer, BertModel

### Load semi structured data

In [2]:
path = '/Users/nicolasdemuynck/BeCode/GNT-ARAI-1.25/Content/00.Use cases/ML6/data/10N_Sodium_Hydroxide_NaOH_40_6_US_EN_sds (1).pdf'
text = tx.process(path)
text = text.decode('utf-8')
#print(text)

### Split text to extract headers

In [3]:
import re

# text = repr(text)
splitted_text = text.split('\n')
#print(splitted_text)

In [4]:
splitted_text[4]

'Safety Data Sheet (SDS)'

In [5]:
print(type(text))
print(type(splitted_text))

<class 'str'>
<class 'list'>


### Create dictionairy key: header , value: corpus

In [6]:
while '' in splitted_text:
    splitted_text.remove('')

index = np.arange(0,len(splitted_text))
df_text = pd.DataFrame(splitted_text, index=[index], columns=['sentence'])
#df_text.head(15)

In [7]:
def check_if_title(sentence):
    sentence = str(sentence)
    word_list = ['identification', 'information', 'measures', 'handling', 'properties', 'stability', 'considerations', 'exposure']
    
    starts_with_section = False
    starts_with_number = False
    if len(sentence.split()) > 1:
        starts_with_section = sentence.split()[0] == 'section'
        starts_with_number = sentence[0].isdigit() and not any(char.isdigit() for char in sentence[2:])
    
    
    contains_title_word = [ele for ele in word_list if(ele in sentence)] 
    
    if (starts_with_number or starts_with_section) and contains_title_word: 
        return 1
    else: 
        return 0

In [8]:
df_text["sentence"] = df_text["sentence"].str.lower()
df_text['title'] = df_text['sentence'].map(lambda a: check_if_title(a))

In [9]:
def dict_titles_with_values(data):
    lst = {}
    current_title = ''
    for i,j in zip(data['title'], data['sentence']):
        if i == 1:
            current_title = j
            lst[j] = []
        else:
            if current_title != '':
                lst[current_title].append(j)
    return lst

In [10]:
df1 = dict_titles_with_values(df_text)

In [11]:
def listToString(s):
    str1 = " " 
    # return string   
    return (str1.join(s))

In [12]:
d2 = {}
for k, v in df1.items():
    d2[k] = listToString(v)

In [13]:
d2

{'1 identification': '· product identifier · trade name: 10n sodium hydroxide (naoh 40%) · product number: ngt-10n naoh · relevant identified uses of the substance or mixture and uses advised against: no further relevant information available. · product description pc21 laboratory chemicals · application of the substance / the mixture: laboratory chemicals · details of the supplier of the safety data sheet: · manufacturer/supplier: nugeneration technologies, llc (dba nugentec) 1155 park avenue, emeryville, ca 94608 salesteam@nugentec.com www.nugentec.com 1-888-996-8436 or 1-707-820-4080 for product information · emergency telephone number: pers emergency response: domestic and canada - 1-800-633-8253, international 1-801-629-0667 *',
 '2 hazard(s) identification': '· classification of the substance or mixture: \uf082 ~ d ghs05 corrosion skin corr. 1a h314 causes severe skin burns and eye damage. eye dam. 1 h318 causes serious eye damage. · label elements: · ghs label elements the produ

In [14]:
var_1 = d2.values()
var_2 = d2.keys()

In [15]:
var_1

dict_values(['· product identifier · trade name: 10n sodium hydroxide (naoh 40%) · product number: ngt-10n naoh · relevant identified uses of the substance or mixture and uses advised against: no further relevant information available. · product description pc21 laboratory chemicals · application of the substance / the mixture: laboratory chemicals · details of the supplier of the safety data sheet: · manufacturer/supplier: nugeneration technologies, llc (dba nugentec) 1155 park avenue, emeryville, ca 94608 salesteam@nugentec.com www.nugentec.com 1-888-996-8436 or 1-707-820-4080 for product information · emergency telephone number: pers emergency response: domestic and canada - 1-800-633-8253, international 1-801-629-0667 *', '· classification of the substance or mixture: \uf082 ~ d ghs05 corrosion skin corr. 1a h314 causes severe skin burns and eye damage. eye dam. 1 h318 causes serious eye damage. · label elements: · ghs label elements the product is classified and labeled according 

In [16]:
var_2

dict_keys(['1 identification', '2 hazard(s) identification', '3 composition/information on ingredients', '4 first-aid measures', '5 fire-fighting measures', '6 accidental release measures', '7 handling and storage', '8 exposure controls/personal protection', '9 physical and chemical properties', '10 stability and reactivity', '11 toxicological information', '12 ecological information', '13 disposal considerations', '14 transport information', '15 regulatory information', '16 other information'])

In [36]:
def text_2_dic(text):
    # Splitting 
    text = text.split('\n')
    text = [item.split(':') for item in text]
    text = [[y.split('=') for y in z] for z in text]
    # List of inner dictionaries
    inner_dic_list = []
    for item in text:
        dic={}
        for a,b in item:
            dic[a] = b
        inner_dic_list.append(dic)
    # Enumeration and looping into a main dictionary
    main_dic = {}
    for i, inner_dic in enumerate(inner_dic_list):
        main_dic[i] = inner_dic
        
    return main_dic

In [44]:
text_2_dic(var_1)

AttributeError: 'dict_values' object has no attribute 'split'

### Make dataframe with column corpus sentences and a column header

In [14]:
def split_text_into_sentences(text):
    import nltk.data
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    doc = nltk.tokenize.sent_tokenize(text)
    return doc

In [15]:
d3 = {}
for k, v in d2.items():
    d3[k] = split_text_into_sentences(v)

In [16]:
d3

{'1 identification': ['· product identifier · trade name: 10n sodium hydroxide (naoh 40%) · product number: ngt-10n naoh · relevant identified uses of the substance or mixture and uses advised against: no further relevant information available.',
  '· product description pc21 laboratory chemicals · application of the substance / the mixture: laboratory chemicals · details of the supplier of the safety data sheet: · manufacturer/supplier: nugeneration technologies, llc (dba nugentec) 1155 park avenue, emeryville, ca 94608 salesteam@nugentec.com www.nugentec.com 1-888-996-8436 or 1-707-820-4080 for product information · emergency telephone number: pers emergency response: domestic and canada - 1-800-633-8253, international 1-801-629-0667 *'],
 '2 hazard(s) identification': ['· classification of the substance or mixture: \uf082 ~ d ghs05 corrosion skin corr.',
  '1a h314 causes severe skin burns and eye damage.',
  'eye dam.',
  '1 h318 causes serious eye damage.',
  '· label elements: · 

In [17]:
dataframe = pd.DataFrame({"corpus": d3.values(), "header": d3.keys()})

In [39]:
df = dataframe.explode('corpus')

In [40]:
def clean_text(text):
    # remove unwanted chars
    text = re.sub(r'[^\w]', ' ', text)
    # remove starting and trailing spaces
    text = text.lstrip()
    text = text.rstrip()
    # make everything lowercase
    text = text.lower()
    # delete multiple spaces
    text = re.sub(' +', ' ', text)
    return text

In [41]:
df['corpus'] = df['corpus'].apply(lambda x: clean_text(x))
df

,corpus,header
0,product identifier trade name 10n sodium hydro...,1 identification
0,product description pc21 laboratory chemicals ...,1 identification
1,classification of the substance or mixture d g...,2 hazard(s) identification
1,1a h314 causes severe skin burns and eye damage,2 hazard(s) identification
1,eye dam,2 hazard(s) identification
...,...,...
15,it is the responsibility of the user to determ...,16 other information
15,date of preparation last revision 02 09 2017 5...,16 other information
15,1a skin corrosion irritation category 1a eye dam,16 other information
15,1 serious eye damage eye irritation category 1...,16 other information


,0,0,1,1,1,1,1,1,1,1,...,14,14,14,15,15,15,15,15,15,15
corpus,product identifier trade name 10n sodium hydro...,product description pc21 laboratory chemicals ...,classification of the substance or mixture d g...,1a h314 causes severe skin burns and eye damage,eye dam,1 h318 causes serious eye damage,label elements ghs label elements the product ...,hazard pictograms d ghs05 signal word danger h...,precautionary statements p260 do not breathe d...,p280 wear protective gloves protective clothin...,...,contd,on page 11 us 44 2 1 page 11 11 safety data sh...,chemical safety assessment a chemical safety a...,the information and recommendations in this sa...,nothing herein shall be deemed to create warra...,it is the responsibility of the user to determ...,date of preparation last revision 02 09 2017 5...,1a skin corrosion irritation category 1a eye dam,1 serious eye damage eye irritation category 1...,sds created by msds authoring services www msd...
header,1 identification,1 identification,2 hazard(s) identification,2 hazard(s) identification,2 hazard(s) identification,2 hazard(s) identification,2 hazard(s) identification,2 hazard(s) identification,2 hazard(s) identification,2 hazard(s) identification,...,15 regulatory information,15 regulatory information,15 regulatory information,16 other information,16 other information,16 other information,16 other information,16 other information,16 other information,16 other information


In [42]:
### not label encoding yet

In [256]:
### train neural network that predicts if a work is key or value

### Label's encoding

In [253]:
list_uniques = list(df.header.unique())
label_dict = dict(enumerate(list_uniques))
label_map = {v: k for k, v in label_dict.items()}

In [254]:
df['id_label'] = df['header'].map(label_map)
df.head(5)

,corpus,header,id_label
0,product identifier trade name 10n sodium hydro...,1 identification,0
0,product description pc21 laboratory chemicals ...,1 identification,0
1,classification of the substance or mixture d g...,2 hazard(s) identification,1
1,1a h314 causes severe skin burns and eye damage,2 hazard(s) identification,1
1,eye dam,2 hazard(s) identification,1


### Split dataset

In [241]:
X = df.corpus
y = df.id_label

In [242]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Tokenization

In [244]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

## Varia

### Tokenization

In [133]:
import spacy
sp = spacy.load('en_core_web_sm')
sentence = sp(var_1)
for word in sentence:
    print(word.text,  word.pos_, word.dep_) #token, part of speech, dependency

product NOUN nmod
identifier ADJ amod
trade NOUN compound
name NOUN compound
10n NOUN compound
sodium NOUN nsubj
hydroxide ADV advmod
naoh VERB ROOT
40 NUM nummod
product NOUN compound
number NOUN nsubj
ngt ADJ appos
10n NOUN npadvmod
naoh VERB ccomp
relevant ADJ amod
identified VERB amod
uses NOUN dobj
of ADP prep
the DET det
substance NOUN pobj
or CCONJ cc
mixture NOUN conj
and CCONJ cc
uses VERB conj
advised VERB advcl
against ADP prep
no DET det
further ADJ amod
relevant ADJ amod
information NOUN nmod
available ADJ amod
product NOUN compound
description NOUN compound
pc21 PROPN meta
laboratory NOUN compound
chemicals NOUN compound
application NOUN pobj
of ADP prep
the DET det
substance NOUN pobj
the DET det
mixture NOUN compound
laboratory NOUN compound
chemicals NOUN compound
details NOUN appos
of ADP prep
the DET det
supplier NOUN pobj
of ADP prep
the DET det
safety NOUN compound
data NOUN compound
sheet NOUN compound
manufacturer NOUN compound
supplier NOUN compound
nugeneration

### Named entity recognition

In [134]:
for entity in sentence.ents:
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))

10n - DATE - Absolute or relative dates or periods
40 - CARDINAL - Numerals that do not fall under another type
10n - DATE - Absolute or relative dates or periods
pc21 - ORG - Companies, agencies, institutions, etc.
94608 - CARDINAL - Numerals that do not fall under another type
1 - CARDINAL - Numerals that do not fall under another type
8436 - DATE - Absolute or relative dates or periods
1 707 - CARDINAL - Numerals that do not fall under another type
4080 - MONEY - Monetary values, including unit
1 - CARDINAL - Numerals that do not fall under another type
633 - CARDINAL - Numerals that do not fall under another type
8253 - DATE - Absolute or relative dates or periods
1 - CARDINAL - Numerals that do not fall under another type


In [44]:
def named_entity_recognition(text):
    nlp = en_core_web_sm.load()
    doc = nlp(text)
    displacy.render(doc, jupyter=True, style='ent')
    return doc

In [18]:
for k, v in d3.items():
    d3[k] = named_entity_recognition(v)

/opt/anaconda3/envs/deep_learning/lib/python3.7/site-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [19]:
# Loop over all the tokens preprocessed by SpaCy
def loop_over_tokens(doc):
    for token in doc:
        # If the token is detected as an entity
        if token.ent_type_:
            print(f"{token.text}: {token.ent_type_}")

In [20]:
for k, v in d3.items():
    d3[k] = loop_over_tokens(v)

10n: DATE
40: CARDINAL
10n: DATE
pc21: ORG
94608: CARDINAL
1: CARDINAL
8436: DATE
1: CARDINAL
707: CARDINAL
4080: MONEY
1: CARDINAL
633: CARDINAL
8253: DATE
1: CARDINAL
1a: CARDINAL
several: TIME
minutes: TIME
2: CARDINAL
44: CARDINAL
2: CARDINAL
1: CARDINAL
2: DATE
11: DATE
29: CARDINAL
1200: CARDINAL
03: CARDINAL
2017: DATE
02: CARDINAL
09: CARDINAL
2017: CARDINAL
10n: DATE
40: CARDINAL
first: ORDINAL
0: CARDINAL
0: CARDINAL
1: CARDINAL
2: CARDINAL
3: CARDINAL
4: CARDINAL
3: CARDINAL
0: CARDINAL
0: CARDINAL
0: CARDINAL
4: CARDINAL
3: CARDINAL
0: CARDINAL
1: CARDINAL
3: CARDINAL
0: CARDINAL
1: CARDINAL
7732: DATE
18: DATE
5: DATE
40: CARDINAL
60: CARDINAL
1310: DATE
2: CARDINAL
25: QUANTITY
50: QUANTITY
skin: QUANTITY
1a: CARDINAL
1910: DATE
1200: DATE
29: CARDINAL
1200: CARDINAL
first: ORDINAL
3: CARDINAL
44: CARDINAL
2: CARDINAL
1: CARDINAL
3: CARDINAL
11: CARDINAL
29: CARDINAL
1200: CARDINAL
03: CARDINAL
2017: DATE
02: CARDINAL
09: CARDINAL
2017: CARDINAL
10n: DATE
40: CARDINAL
sev

### detecting nouns

In [136]:
for noun in sentence.noun_chunks:
    print(noun.text)

product identifier trade name 10n sodium
40 product number
relevant identified uses
the substance
mixture
no further relevant information available product description pc21 laboratory chemicals application
the substance
the mixture laboratory chemicals details
the supplier
the safety data sheet manufacturer supplier nugeneration technologies
dba nugentec 1155 park avenue emeryville
94608 salesteam nugentec com www nugentec
product information emergency telephone number
emergency response


In [140]:
var_1

'product identifier trade name 10n sodium hydroxide naoh 40 product number ngt 10n naoh relevant identified uses of the substance or mixture and uses advised against no further relevant information available product description pc21 laboratory chemicals application of the substance the mixture laboratory chemicals details of the supplier of the safety data sheet manufacturer supplier nugeneration technologies llc dba nugentec 1155 park avenue emeryville ca 94608 salesteam nugentec com www nugentec com 1 888 996 8436 or 1 707 820 4080 for product information emergency telephone number pers emergency response domestic and canada 1 800 633 8253 international 1 801 629 0667'

### part of speech tagging

In [139]:
for token in sentence:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

product product NOUN NN nmod xxxx True False
identifier identifi ADJ JJR amod xxxx True False
trade trade NOUN NN compound xxxx True False
name name NOUN NN compound xxxx True True
10n 10n NOUN NNS compound ddx False False
sodium sodium NOUN NN nsubj xxxx True False
hydroxide hydroxide ADV RB advmod xxxx True False
naoh naoh VERB VBD ROOT xxxx True False
40 40 NUM CD nummod dd False False
product product NOUN NN compound xxxx True False
number number NOUN NN nsubj xxxx True False
ngt ngt ADJ JJ appos xxx True False
10n 10n NOUN NNS npadvmod ddx False False
naoh naoh VERB VBD ccomp xxxx True False
relevant relevant ADJ JJ amod xxxx True False
identified identify VERB VBN amod xxxx True False
uses use NOUN NNS dobj xxxx True False
of of ADP IN prep xx True True
the the DET DT det xxx True True
substance substance NOUN NN pobj xxxx True False
or or CCONJ CC cc xx True True
mixture mixture NOUN NN conj xxxx True False
and and CCONJ CC cc xxx True True
uses use VERB VBZ conj xxxx True False

### Removing stop words

In [145]:
from spacy.lang.en import English
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

#  "nlp" Object is used to create documents with linguistic annotations.
my_doc = nlp(var_1)
# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)

from spacy.lang.en.stop_words import STOP_WORDS
# Create list of word tokens after removing stopwords
filtered_sentence =[] 
for word in token_list:
    lexeme = nlp.vocab[word]
if lexeme.is_stop == False:
    filtered_sentence.append(word) 
    print(token_list)
    print(filtered_sentence)   

['product', 'identifier', 'trade', 'name', '10n', 'sodium', 'hydroxide', 'naoh', '40', 'product', 'number', 'ngt', '10n', 'naoh', 'relevant', 'identified', 'uses', 'of', 'the', 'substance', 'or', 'mixture', 'and', 'uses', 'advised', 'against', 'no', 'further', 'relevant', 'information', 'available', 'product', 'description', 'pc21', 'laboratory', 'chemicals', 'application', 'of', 'the', 'substance', 'the', 'mixture', 'laboratory', 'chemicals', 'details', 'of', 'the', 'supplier', 'of', 'the', 'safety', 'data', 'sheet', 'manufacturer', 'supplier', 'nugeneration', 'technologies', 'llc', 'dba', 'nugentec', '1155', 'park', 'avenue', 'emeryville', 'ca', '94608', 'salesteam', 'nugentec', 'com', 'www', 'nugentec', 'com', '1', '888', '996', '8436', 'or', '1', '707', '820', '4080', 'for', 'product', 'information', 'emergency', 'telephone', 'number', 'pers', 'emergency', 'response', 'domestic', 'and', 'canada', '1', '800', '633', '8253', 'international', '1', '801', '629', '0667']
['0667']


In [23]:
var_1

NameError: name 'var_1' is not defined